In [1]:
import torch
import numpy as np
from torch.nn import CrossEntropyLoss
from torch.utils.data import Dataset

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score


2025-12-31 10:08:21.598971: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767175701.787471      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767175701.843768      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767175702.303168      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767175702.303225      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767175702.303229      55 computation_placer.cc:177] computation placer alr

In [2]:
import gdown
import pandas as pd

# -------- Train --------
train_id = "1GKQt7iAbiMfwWMyNXXTkO8WCRA6XdeJgqP9ip1koeQo"
train_file = "train.xlsx"
train_url = f"https://docs.google.com/spreadsheets/d/{train_id}/export?format=xlsx"

gdown.download(train_url, train_file, quiet=False)
train_df = pd.read_excel(train_file)


# -------- Validation --------
val_id = "1JIrWLlWDgKeWjkfHpnlayNAiLinS-TcRmL2oSp4oZsQ"
val_file = "validation.xlsx"
val_url = f"https://docs.google.com/spreadsheets/d/{val_id}/export?format=xlsx"

gdown.download(val_url, val_file, quiet=False)
val_df = pd.read_excel(val_file)


# -------- Test --------
test_id = "1COiSxsK0HKbhLHM-up2UZngZ_neZV7OwZbZJjOw6M4g"
test_file = "test.xlsx"
test_url = f"https://docs.google.com/spreadsheets/d/{test_id}/export?format=xlsx"

gdown.download(test_url, test_file, quiet=False)
test_df = pd.read_excel(test_file)


train_df.head(), val_df.head(), test_df.head()


/usr/local/lib/python3.12/dist-packages/gdown/parse_url.py:48: UserWarning: You specified a Google Drive link that is not the correct link to download a file. You might want to try `--fuzzy` option or the following url: https://drive.google.com/uc?id=None
  warnings.warn(
Downloading...
From: https://docs.google.com/spreadsheets/d/1GKQt7iAbiMfwWMyNXXTkO8WCRA6XdeJgqP9ip1koeQo/export?format=xlsx
To: /kaggle/working/train.xlsx
475kB [00:00, 9.82MB/s]
/usr/local/lib/python3.12/dist-packages/gdown/parse_url.py:48: UserWarning: You specified a Google Drive link that is not the correct link to download a file. You might want to try `--fuzzy` option or the following url: https://drive.google.com/uc?id=None
  warnings.warn(
Downloading...
From: https://docs.google.com/spreadsheets/d/1JIrWLlWDgKeWjkfHpnlayNAiLinS-TcRmL2oSp4oZsQ/export?format=xlsx
To: /kaggle/working/validation.xlsx
122kB [00:00, 5.35MB/s]
/usr/local/lib/python3.12/dist-packages/gdown/parse_url.py:48: UserWarning: You specified a

(                                                text  label
 0  এই গেরুয়া হিন্দুদের জন্য একজন সাধারণ হিন্দু হি...      0
 1  একটা উপদেশঃ আন্দোলন চালিয়ে যান,,,যতক্ষন না পর্...      0
 2  বাংলাদেশে ধর্মীয় ইস্যুতে কোন হিন্দুর বিচার কী ...      0
 3  স্কুল কলেজ ইউনিভার্সিটি39তে সরস্বতী পূজা হতে প...      0
 4  আমাদের দেশে পুলিশ এত ক্ষমতা পায় কোথায় থেকে এর...      0,
                                                 text  label
 0  চিটার বাটপার বাটপার সব সময় টিভি কালকে সারা দি...      1
 1  যেদিন বাঘের নাম বাদ দিয়ে বন বিড়াল রাখা হবে সেদ...      1
 2          পহেলা বৈশাখের তিন ধাপের নিরাপত্তা কোথায়?      0
 3  ব্যবসাহিক রা আসলে একটু বেশি বাটপার বেশি বাড়াব...      1
 4  ভাগ্যিস মানুষের ঘরে জন্ম নিয়েছি, আওয়ামী লীগের ...      0,
                                                 text  label
 0          যারা মারছে তাদের বিচার হোক ফোটেজ দেখে হোক      0
 1                       ভুট তুদের পিচন দিয়ে দেয়া হবে      2
 2  ষড়যন্ত্রকারীরা কত শক্তিশালী! অঘটন ঘটানোর জন্য...      0
 3  এরকম শিক্ষা প্রথিস

In [3]:
MODEL_NAME = "FacebookAI/xlm-roberta-large"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3
)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# create dataset classes

MAX_LEN = 256  # max token length for Bangla sentences

class ViolenceDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=MAX_LEN):
        self.texts = df['text'].tolist()
        self.labels = df['label'].tolist()
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = int(self.labels[idx])

        encoding = self.tokenizer(
            text,
            max_length=self.max_len,
            # padding='max_length',
            padding=False,
            truncation=True,
            return_tensors='pt',
            add_special_tokens=True
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Create dataset objects
train_dataset = ViolenceDataset(train_df, tokenizer)
val_dataset = ViolenceDataset(val_df, tokenizer)

In [5]:
# Compute class weights
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_df['label']),
    y=train_df['label']
)

class_weights = torch.tensor(class_weights, dtype=torch.float)

In [6]:
# Define compute_metrics function

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)

    return {
        "accuracy": accuracy_score(labels, preds),
        "f1_macro": f1_score(labels, preds, average="macro"),
        "precision_macro": precision_score(labels, preds, average="macro", zero_division=0),
        "recall_macro": recall_score(labels, preds, average="macro", zero_division=0),
    }

In [7]:
# Define TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=8,  
    max_steps=-1,

    learning_rate=5e-6,  # Lower LR (1e-5) is frequently reported as optimal for XLM-RoBERTa fine-tuning on Bangla tasks
    weight_decay=0.01,  # Standard value; kept moderate to prevent overfitting
    warmup_ratio=0.06,  # Common default (~6% warmup); adjusts to longer epochs

    per_device_train_batch_size=8,  # Increased to 16 (effective batch ~32 with accumulation); better gradient stability for XLM-R
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,

    eval_strategy="steps",
    eval_steps=200,

    logging_strategy="steps",
    logging_steps=20,
    logging_first_step=True,

    save_strategy="steps",
    save_steps=200,
    save_total_limit=2,

    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    greater_is_better=True,

    fp16=torch.cuda.is_available(),
    max_grad_norm=1.0,

    report_to="none",
    disable_tqdm=False,
)

In [8]:
# Custom loss with class weights

class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels").long()
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fn = CrossEntropyLoss(
            weight=class_weights.to(logits.device)
        )
        loss = loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [9]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer)


In [10]:
# Initialize Trainer

trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

/tmp/ipykernel_55/3201740747.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  trainer = WeightedTrainer(


In [ ]:
trainer.train()

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Get validation predictions
val_predictions = trainer.predict(val_dataset)
val_preds = np.argmax(val_predictions.predictions, axis=1)
val_labels = val_predictions.label_ids

# Compute confusion matrix
cm_val = confusion_matrix(val_labels, val_preds)

# Display
plt.figure(figsize=(5,4))
sns.heatmap(cm_val, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Validation Confusion Matrix')
plt.show()

In [ ]:
# convert test data

test_dataset = ViolenceDataset(test_df, tokenizer)

In [ ]:
# Get predictions from the trained model

predictions = trainer.predict(test_dataset)
preds = np.argmax(predictions.predictions, axis=1)

In [ ]:
# Add a new column for predicted labels
test_df['predicted_label'] = preds

# Save to CSV
test_df.to_csv("bangla_violence_test_predictions.csv", index=False)

# Check first few rows
print(test_df.head())

In [ ]:

from sklearn.metrics import classification_report

predictions = trainer.predict(test_dataset)

y_pred = np.argmax(predictions.predictions, axis=1)

y_true = predictions.label_ids


In [ ]:
print(
    classification_report(
        y_true,
        y_pred,
        target_names=["class_0", "class_1", "class_2"],
        digits=2
    )
)


In [ ]:
# Confusion Matrix

if 'label' in test_df.columns:
    test_labels = test_df['label'].values
    cm_test = confusion_matrix(test_labels, preds)

    plt.figure(figsize=(5,4))
    sns.heatmap(cm_test, annot=True, fmt='d', cmap='Greens')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Test Confusion Matrix')
    plt.show()
else:
    print("No true labels in test set; cannot compute confusion matrix for test data.")

In [ ]:
print(
    classification_report(
        y_true,
        y_pred,
        target_names=["class_0", "class_1", "class_2"],
        digits=2
    )
)